In [ ]:
import cv2
import mediapipe as mp
from flask import Flask, request, jsonify


In [ ]:
app = Flask(__name__)

@app.route('/detect_emotion', methods=['POST'])
def detect_emotion():
    # Get the image from the request
    image = request.files['image'].read()

    # Perform emotion detection on the image
    result = perform_emotion_detection(image)

    # Return the result as JSON
    return jsonify(result)


In [ ]:
def perform_emotion_detection(image):
    # Convert the image data into OpenCV format
    image = cv2.imdecode(np.frombuffer(image, np.uint8), cv2.IMREAD_COLOR)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Initialize Mediapipe holistic model
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        # Process the image
        results = holistic.process(image_rgb)

        # Retrieve the face landmarks
        face_landmarks = results.face_landmarks

        # Perform emotion detection based on facial expressions
        if face_landmarks is not None:
            # Calculate the distances between key facial landmarks
            eye_distance = calculate_distance(face_landmarks.landmark[mp_holistic.FaceLandmark.LEFT_EYE].x,
                                              face_landmarks.landmark[mp_holistic.FaceLandmark.LEFT_EYE].y,
                                              face_landmarks.landmark[mp_holistic.FaceLandmark.RIGHT_EYE].x,
                                              face_landmarks.landmark[mp_holistic.FaceLandmark.RIGHT_EYE].y)
            mouth_distance = calculate_distance(face_landmarks.landmark[mp_holistic.FaceLandmark.MOUTH_LEFT].x,
                                                face_landmarks.landmark[mp_holistic.FaceLandmark.MOUTH_LEFT].y,
                                                face_landmarks.landmark[mp_holistic.FaceLandmark.MOUTH_RIGHT].x,
                                                face_landmarks.landmark[mp_holistic.FaceLandmark.MOUTH_RIGHT].y)

            # Perform emotion detection based on the distances
            if eye_distance > threshold and mouth_distance > threshold:
                emotion = 'happy'
            else:
                emotion = 'sad'
        else:
            emotion = 'unknown'

        # Retrieve the body landmarks
        body_landmarks = results.pose_landmarks

        # Perform emotion detection based on body movements
        if body_landmarks is not None:
            # Calculate the average y-coordinate of the body landmarks
            avg_y = np.mean([landmark.y for landmark in body_landmarks.landmark])

            # Perform emotion detection based on the average y-coordinate
            if avg_y > threshold:
                emotion = 'excited'
            else:
                emotion = 'angry'

    # Format the results
    emotions = {
        'emotion': emotion
    }

    return emotions


def calculate_distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)


In [ ]:
if __name__ == '__main__':
    app.run()


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
pip install mediapipe opencv-python flask


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 29.5 MB/s eta 0:00:00
